In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain.agents import initialize_agent, Tool

In [2]:

from langchain.agents import create_csv_agent
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType

csv_agent = create_csv_agent(
  ChatOpenAI(temperature=0, model="gpt-4"),
  ["clientes.csv", "produtos.csv", "vendas.csv"],
  verbose=True,
  agent_type=AgentType.OPENAI_FUNCTIONS  
)

# query("What is the email of Pearl Johnston?")

In [3]:
# Create tools
tools = [
  Tool.from_function(
    name="Search",
    description="Useful when you need to answer questions about a clients (clientes.csv: id,name,email,address) and sales (vendas.csv: id,productId,customerId,timestamp). Input should be in the form of a question containing full context.",
    func=csv_agent.run
  )
]

In [4]:
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

# import langchain
# langchain.debug = True

In [5]:
# mrkl.run("What is the weather in LA and SF?")
# mrkl.run("What is email of Pearl Johnston?")
# agent.run("What is name of Cody1@yahoo.com?")
# agent.run("What is the name and email of customer with more purchases?")

agent.run("Which products are purchased by customer with email Jazmyne67@yahoo.com?")



> Entering new AgentExecutor chain...
{
    "action": "Search",
    "action_input": "Which products are purchased by customer with email Jazmyne67@yahoo.com?"
}

> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `customer_id = df1.loc[df1['email'] == 'Jazmyne67@yahoo.com', 'id'].values[0]
customer_id`
responded: {content}

175
Invoking: `python_repl_ast` with `product_ids = df3.loc[df3['customerId'] == customer_id, 'productId'].values
product_ids`
responded: {content}

[ 9 17 14 10 17  5]
Invoking: `python_repl_ast` with `product_names = df2.loc[df2['id'].isin(product_ids), 'name'].values
product_names`
responded: {content}

['Generic Wooden Shirt' 'Luxurious Steel Car' 'Awesome Cotton Bacon'
 'Licensed Frozen Bacon' 'Handmade Rubber Pants']The customer with the email "Jazmyne67@yahoo.com" has purchased the following products:

- Generic Wooden Shirt
- Luxurious Steel Car
- Awesome Cotton Bacon
- Licensed Frozen Bacon
- Handmade Rubber Pants

> Finished chain.



"The customer with the email 'Jazmyne67@yahoo.com' has purchased the following products: Generic Wooden Shirt, Luxurious Steel Car, Awesome Cotton Bacon, Licensed Frozen Bacon, Handmade Rubber Pants."

In [6]:
agent.run("How much this customer spend?")



> Entering new AgentExecutor chain...
{
    "action": "Search",
    "action_input": "How much did customer Jazmyne67@yahoo.com spend?"
}

> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `customer_id = df1[df1['email'] == 'Jazmyne67@yahoo.com']['id'].values[0]
customer_id`
responded: {content}

175
Invoking: `python_repl_ast` with `purchased_products = df3[df3['customerId'] == customer_id]['productId'].values
purchased_products`
responded: {content}

[ 9 17 14 10 17  5]
Invoking: `python_repl_ast` with `total_spent = df2[df2['id'].isin(purchased_products)]['price'].sum()
total_spent`
responded: {content}

3550The customer "Jazmyne67@yahoo.com" spent a total of 3550.

> Finished chain.

Observation: The customer "Jazmyne67@yahoo.com" spent a total of 3550.
Thought:{
    "action": "Final Answer",
    "action_input": "The customer 'Jazmyne67@yahoo.com' spent a total of 3550."
}

> Finished chain.


"The customer 'Jazmyne67@yahoo.com' spent a total of 3550."